In [3]:
import numpy as np
import pandas as pd
import re, os
from pathlib import Path

# Eliminate any updated_* in the cwd
deleted_files = [p.unlink() for p in Path.cwd().glob('updated_*') if p.is_file()]

print(f"Done! Deleted {len(deleted_files)} file(s).")

# Previous code start from here.
all_files = os.listdir()
bcourse = '.*_Grades-CIVENG_93.csv'
iclicker = 'iClicker_GradesExport_Canvas_.*.csv'

for i in all_files:
	if re.match(bcourse, i):
		bcourse = i
	if re.match(iclicker, i):
		iclicker = i

ic = pd.read_csv(iclicker)
bc = pd.read_csv(bcourse)
new_bc = bc.copy(); print(new_bc.columns);
def poll_name():
	search_type = [i for i in ic.columns if re.match('Class \d+ - Poll', i)][0]
	return [i for i in bc.columns if re.match(search_type+' .*', i)][0]

for i in bc.iterrows():
	stu = ic[ic['SIS User ID'] == i[1]['SIS User ID']]; total_score = 0 # Start sum of the total_score
	if (stu['Student'].values.size > 0) and (stu['Current Score'].values.size > 0):
		# if stu['Student'].values[0]!=i[1]['Student']:
		# 	print(stu['Student'].values[0], i[1]['Student'])
		poll_num = list(i[1].index).index([j for j in i[1].index if poll_name() in j][0])
		max_score = max(float(stu['Current Score'].values[0]), i[1][poll_num]);total_score +=max_score # Sum the total
		print(i[1]['Student'], max_score)
		idx = new_bc[new_bc['SIS User ID'] == i[1]['SIS User ID']].index
		new_bc.at[idx.values[0], poll_name()] = str(max_score); new_bc.at[idx.values[0], 'Final Score'] = str(total_score)
		print('Previous', i[1][poll_num])
		print('updated', new_bc[new_bc['SIS User ID'] == i[1]['SIS User ID']][poll_name()]); print('updated', new_bc[new_bc['SIS User ID'] == i[1]['SIS User ID']]['Final Score'])
u = list(new_bc.columns[:5])+[poll_name(), "Final Score"]
new_bc[u].to_csv('updated_'+bcourse, encoding='utf-8-sig', index=False)

Done! Deleted 1 file(s).
Index(['Student', 'ID', 'SIS User ID', 'SIS Login ID', 'Section',
       'Class 1 - Poll (8935383)', 'iClicker (Total) (8935426)',
       'Homework Assignments Current Score',
       'Homework Assignments Unposted Current Score',
       'Homework Assignments Final Score',
       'Homework Assignments Unposted Final Score',
       'Lab Assignments Current Score',
       'Lab Assignments Unposted Current Score', 'Lab Assignments Final Score',
       'Lab Assignments Unposted Final Score', 'Quizzes Current Score',
       'Quizzes Unposted Current Score', 'Quizzes Final Score',
       'Quizzes Unposted Final Score', 'Midterm Exams Current Score',
       'Midterm Exams Unposted Current Score', 'Midterm Exams Final Score',
       'Midterm Exams Unposted Final Score', 'Projects Current Score',
       'Projects Unposted Current Score', 'Projects Final Score',
       'Projects Unposted Final Score', 'Final Exam Current Score',
       'Final Exam Unposted Current Score',

/tmp/ipykernel_123/2329865068.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_score = max(float(stu['Current Score'].values[0]), i[1][poll_num]);total_score +=max_score # Sum the total
/tmp/ipykernel_123/2329865068.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_bc.at[idx.values[0], poll_name()] = str(max_score); new_bc.at[idx.values[0], 'Final Score'] = str(total_score)
/tmp/ipykernel_123/2329865068.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by po